In [1]:
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from itertools import combinations
from scipy.linalg import cho_solve, cho_factor
from tqdm import tqdm

np.set_printoptions(precision=4, linewidth=200, sign=" ")

## Useful functions from pose_graph_jacobian.ipynb
Just functions for `Exp` and `Log` maps, `cross` product matrix function and calculation of the jacobian

In [2]:
def np_cross(x):
    return np.array([
        [0,-x[2],x[1]],
        [x[2],0,-x[0]],
        [-x[1],x[0],0]
    ])

def np_exp(x):
    theta = np.linalg.norm(x[3:])
    if theta == 0:
        R = np.eye(3)
        t = x[:3]
    else:
        a = np.sin(theta)/theta
        b = (1 - np.cos(theta))/(theta*theta)
        c = (theta - np.sin(theta))/(theta**3)
        w = np_cross(x[3:])
        R = np.eye(3) + a*w + b*w@w

        t = (np.eye(3) + b*w + c*w@w)@x[:3]
    
    mat = np.eye(4)
    mat[:3, :3] = R
    mat[:3, 3] = t

    return mat

def np_log(T):
    theta = np.arccos((np.trace(T) - 2) / 2)
    # theta = np.arccos((T[0, 0] + T[1, 1] + T[2, 2] - 1) / 2)

    if theta == 0:
        return T[0, 3], T[1, 3], T[2, 3], 0, 0, 0

    a = np.sin(theta)/theta
    b = (1 - np.cos(theta))/(theta*theta)

    x4 = 1/(2*a)*(T[2, 1] - T[1, 2])
    x5 = 1/(2*a)*(T[0, 2] - T[2, 0])
    x6 = 1/(2*a)*(T[1, 0] - T[0, 1])
    
    w = np_cross(np.array([x4,x5,x6]))
    u = (np.eye(3) - 1/2*w + 1/(theta**2)*(1 - a/(2*b))*w@w)@T[:3, 3]

    return np.array([u[0], u[1], u[2], x4, x5, x6])

In [3]:
def np_adjoint(X):
    """X is a 4x4 matrix not a 6 Vector"""
    mat = np.zeros((6,6))
    mat[:3,:3] = X[:3,:3]
    mat[3:,3:] = X[:3,:3]
    mat[:3,3:] = np_cross(X[:3,3])@X[:3,:3]

    return mat.T

def np_error(Z, Xi, Xj):
    return np_log(np.linalg.inv(np_exp(Z))@np.linalg.inv(np_exp(Xi))@np_exp(Xj))

def np_exp_error(Z, Xi, Xj):
    return np.linalg.inv(np_exp(Z))@np.linalg.inv(np_exp(Xi))@np_exp(Xj)

In [4]:
def box_plus(a, b):
    return np_log(np_exp(a)@np_exp(b))

def box_minus(a, b):
    return np_log(np.linalg.inv(np_exp(b))@np_exp(a))

In [5]:
def Je_analytic_Xi(Z, Xi, Xj):
    eXi, eXj = np_exp(Xi), np_exp(Xj)
    A = np.zeros((6,6))
    A[:3,:3] = (eXj[:3,:3]).T
    A[3:,3:] = (eXj[:3,:3]).T
    A[:3,3:] = -(eXj[:3,:3]).T@np_cross(eXj[:3,3])

    B = np.zeros_like(A)
    B[:3, :3] = eXi[:3,:3]
    B[3:,3:] = eXi[:3,:3]
    B[:3,3:] = np_cross(eXi[:3, 3])@eXi[:3,:3]

    return -A@B

def Je_analytic_Xj(Z, Xi, Xj):
    return np.eye(6)

In [6]:
def Je_numeric_Xi(Z, Xi, Xj, eps=1E-8, type_=np.float64):    
    Z, Xi, Xj = Z.astype(type_), Xi.astype(type_), Xj.astype(type_)

    generators = [np.array([i==j for i in range(6)], dtype=type_) for j in range(6)]

    error_inv = np.linalg.inv(np_exp_error(Z, Xi, Xj))
    # gen_error = [np.linalg.inv(np_exp(Z))@np.linalg.inv(np_exp(Xi)@np_exp(g_i*eps))@np_exp(Xj) for g_i in generators]
    gen_error = [np.linalg.inv(np_exp(Z))@np.linalg.inv(np_exp(Xi + g_i*eps))@np_exp(Xj) for g_i in generators]


    cols = (1/eps)*np.array([np_log(error_inv@gen_error_i) for gen_error_i in gen_error]).T

    return cols


def Je_numeric_Xj(Z, Xi, Xj, eps=1E-8, type_=np.float64):    
    Z, Xi, Xj = Z.astype(type_), Xi.astype(type_), Xj.astype(type_)

    generators = [np.array([i==j for i in range(6)], dtype=type_) for j in range(6)]

    error_inv = np.linalg.inv(np_exp_error(Z, Xi, Xj))
    # gen_error = [np.linalg.inv(np_exp(Z))@np.linalg.inv(np_exp(Xi))@(np_exp(Xj)@np_exp(g_i*eps))for g_i in generators]
    gen_error = [np.linalg.inv(np_exp(Z))@np.linalg.inv(np_exp(Xi))@np_exp(Xj + g_i*eps) for g_i in generators]


    cols = (1/eps)*np.array([np_log(error_inv@gen_error_i) for gen_error_i in gen_error]).T

    return cols

## Setting up the pose graph problem
The plan here is to define a pose graph problem that is simple to solve and has a known correct solution - i.e. I can check that I'm converging on the right spot. To start with I think I'm going to go with 3 fully connected poses. I then calculate the correct measurements between them as my measurements, and set the information matrices to something reasonable. After that, displace the points and use these new points as my starting off point for the optimisation - if it correctly comes back to the original position then I'll know I've done something right and can move onto progressively harder problems.

In [7]:
n = 3
poses = [np_log(np_exp((np.random.random(6) - 0.5)*2)) for _ in range(n)]
measurements = {(i,j):np_log(np.linalg.inv(np_exp(poses[i]))@np_exp(poses[j])) for (i, j) in combinations(range(n), 2)}

eps = 0.0001
disturbed_poses = [p + np.random.normal(0, eps, 6) if i else p for i, p in enumerate(poses)]

infos = {k:np.linalg.inv(np.eye(6)*0.05) for k in measurements.keys()}

disturbed_poses, poses

([array([ 2.4627e-01,  6.1599e-01, -8.1145e-01, -2.7292e-01,  9.1251e-01,  2.5081e-04]),
  array([ 0.6409,  0.811 , -0.161 , -0.9197, -0.4636, -0.4504]),
  array([ 0.832 ,  0.3071, -0.2176, -0.2669, -0.8683, -0.4583])],
 [array([ 2.4627e-01,  6.1599e-01, -8.1145e-01, -2.7292e-01,  9.1251e-01,  2.5081e-04]),
  array([ 0.6409,  0.8109, -0.1612, -0.9197, -0.4634, -0.4504]),
  array([ 0.832 ,  0.3069, -0.2177, -0.267 , -0.8682, -0.4582])])

Now that we have our poses, measurements and information matrices its time to get on with writing the actual optimisation code. We're going to use Gauss-Newton I think so the update rule is going to look like this:

$$\Delta x = (J^TJ)^{-1}J^Tr$$

In [8]:
def make_parts(poses, measurements, infos):
    H = np.zeros((6*len(poses), 6*len(poses)))
    b = np.zeros(6*len(poses))

    H[:6, :6] += np.eye(6)

    for (i, j), Zij in measurements.items():
        Xi, Xj = poses[i], poses[j]
        omega = infos[(i, j)]

        e = np_error(Zij, Xi, Xj)
        A = Je_numeric_Xi(Zij, Xi, Xj)
        B = Je_numeric_Xj(Zij, Xi, Xj)

        # print(A)
        # print(B)

        H[6*i:6*i+6, 6*i:6*i+6] += A.T@omega@A
        H[6*i:6*i+6, 6*j:6*j+6] += A.T@omega@B
        H[6*j:6*j+6, 6*i:6*i+6] += B.T@omega@A
        H[6*j:6*j+6, 6*j:6*j+6] += B.T@omega@B

        b[6*i:6*i+6] += A.T@omega@e
        b[6*j:6*j+6] += B.T@omega@e

    return H, b

H, b = make_parts(disturbed_poses, measurements, infos)

H.shape, b.shape


((18, 18), (18,))

In [9]:
new_poses = disturbed_poses[:]
print([p for p in new_poses])

for i in tqdm(range(12)):
    H, b = make_parts(new_poses, measurements, infos)
    with np.printoptions(precision=4, suppress=True):
        print(H)
    delta = cho_solve(cho_factor(H), b)

    pose_vector = np.concatenate(new_poses)
    for i in range(0, pose_vector.shape[0]//6):
        # new_poses[i] = np_log(np_exp(new_poses[i])@np_exp(delta[6*i:6*i + 6]))
        new_poses[i] = np_log(np.linalg.inv(np_exp(delta[6*i:6*i + 6]))@np_exp(new_poses[i]))
        # new_poses[i] += delta[6*i:6*i+6]
    print([p for p in new_poses])


# print(new_poses)
# print(poses)
    

[array([ 2.4627e-01,  6.1599e-01, -8.1145e-01, -2.7292e-01,  9.1251e-01,  2.5081e-04]), array([ 0.6409,  0.811 , -0.161 , -0.9197, -0.4636, -0.4504]), array([ 0.832 ,  0.3071, -0.2176, -0.2669, -0.8683, -0.4583])]


  0%|          | 0/12 [00:00<?, ?it/s]

[[ 38.307   -0.8055  -0.0002  -1.7729  27.0772   1.608  -14.0107  -7.2283   9.8964   2.5729  -1.8992   3.031  -11.3093  -4.0261  14.0665   1.0484  -5.7612  -0.0075]
 [ -0.8055  40.7591   0.0007 -26.8676   0.222   20.9071   4.1175 -17.2685  -6.8218  -3.7224   0.9674  -8.3978   5.1553 -18.837   -1.7654   0.131    0.0168  -8.835 ]
 [ -0.0002   0.0007  38.0661  -0.8916 -23.3013  -1.5523 -12.2342   2.1242 -14.185   -4.6986   7.4387   3.7712 -13.8679  -3.4874 -12.1431  -1.1311   6.137   -0.1918]
 [ -1.7729 -26.8676  -0.8916  22.2608  -1.9646 -13.7989  -3.871    9.8929   0.0357   0.8179   1.1521   4.9952   0.0037  16.8158   3.6543   0.1119  -1.2074   7.3944]
 [ 27.0772   0.222  -23.3013  -1.9646  36.5516   3.972    0.8458  -5.8551  15.4691   4.703   -6.3079  -1.1052  -2.3167  -2.1114  19.172    1.5595  -8.5095  -0.11  ]
 [  1.608   20.9071  -1.5523 -13.7989   3.972   14.3345   5.4069  -9.1054  -5.192   -2.4673   0.5954  -5.6846  -0.2591  -9.6364   3.685    0.4045  -1.882   -4.0472]
 [-14.0107

LinAlgError: 18-th leading minor of the array is not positive definite

## Visualising the Optimisation
The plan is to visualise the optimisation process using open3d and some coloured dots.
- Red is the initial "bad" position of the 

In [ ]:
import open3d as o3d
from itertools import chain

In [ ]:
def create_axis(transform, colour, size=0.2):
    mesh = o3d.geometry.TriangleMesh.create_coordinate_frame(size=size, origin=np.array([0.0,0.0,0.0]))
    mesh.paint_uniform_color(colour)
    mesh.transform(transform)
    return mesh

In [ ]:
true_poses = [create_axis(np_exp(x), [0,1,0]) for x in poses]
start_poses = [create_axis(np_exp(x), [1,0,0]) for x in disturbed_poses]
end_poses = [create_axis(np_exp(x), [0,0,1], 0.3) for x in new_poses]

# o3d.visualization.draw_geometries([x for x in chain(start_poses, true_poses, end_poses)])

In [ ]:
def create_error(start, error, colour, radius=0.005, resolution=20):
    end = start@error
    # Extract translation vectors from the matrices
    start_translation = np.array(start[:3, 3])
    end_translation = np.array(end[:3, 3])

    # Calculate the height of the cylinder
    height = np.linalg.norm(end_translation - start_translation)

    # Create a cylinder
    cylinder = o3d.geometry.TriangleMesh.create_cylinder(radius=radius, height=height, resolution=resolution)

    # Calculate the rotation matrix to align the cylinder with the direction from start to end
    z = [0,0,1]
    direction = end_translation - start_translation
    dz = direction/np.linalg.norm(direction)
    dx = np.cross(dz, z)/np.linalg.norm(np.cross(dz, z))
    dy = np.cross(dz, dx)/np.linalg.norm(np.cross(dz, dx))

    rotation_matrix = np.vstack([dx,dy,dz]).T
    print(rotation_matrix)
    print(dx, dy, dz)

    # Apply the rotation and translation to the cylinder
    cylinder.translate(np.array([0, 0, height/2]))
    cylinder.rotate(rotation_matrix, center=(0,0,0))
    cylinder.translate(start_translation)
    cylinder.paint_uniform_color(colour)

    return cylinder

errors = [create_error(np_exp(poses[i]), np_exp(Zij), [0.7,0.7,0.7]) for (i, j), Zij in measurements.items()]

o3d.visualization.draw_geometries([x for x in chain(errors, start_poses, true_poses, end_poses, [create_axis(np.eye(4), [0,0,0])])])

[[-0.9754  0.1219  0.1839]
 [-0.2206 -0.539  -0.8129]
 [ 0.     -0.8334  0.5527]]
[-0.9754 -0.2206  0.    ] [ 0.1219 -0.539  -0.8334] [ 0.1839 -0.8129  0.5527]
[[-0.3358  0.8739  0.3515]
 [-0.9419 -0.3116 -0.1253]
 [ 0.     -0.3731  0.9278]]
[-0.3358 -0.9419  0.    ] [ 0.8739 -0.3116 -0.3731] [ 0.3515 -0.1253  0.9278]
[[ 0.9679  0.1241  0.2185]
 [-0.2512  0.478   0.8417]
 [ 0.     -0.8696  0.4938]]
[ 0.9679 -0.2512  0.    ] [ 0.1241  0.478  -0.8696] [ 0.2185  0.8417  0.4938]
